# Activities

In this practicum the functionality of the console application that have been built in practicum 1 and 2 will be extended. We discussed ealier the MVC architecture.  After implementing the `model` part that is resposible for the commuication with the data storage we will add a `controller` part to the application. This controller will serve as a HTTP application programming interface (API). So it is just an object that handles HTTP requests and creates the HTTP response. Any addtional logic that is needed to implement a response will be implmented in the controller. 

In this example we will:

- Build a single RESTful controller against an EF database
- Add pagination to `GetAll`
- Add very simple optional filtering to `GetAll` (`GreaterThan`)
- Add very simple optional sorting to `GetAll` (by a single property)
- Create filtering criteria dynamically by building `Linq expressions`. This criteria will be deserialed from a json message.  

### Step 0:  Project setup

- To start the implementation you need two addtional tools: 
    - A HTTP client named `Postman` that send and recieve HTTP messages (json) to and from a web API.  Download Postman:(https://www.getpostman.com/).

    - A .net core package called `Newtonsoft.Json` to serialize json messages and deserialize .NET object. This package will be added after project creation.


- In order to autmatically generate a controller in dotnet core we will create a `webapi-application`.  Then we add the `model` part to it (see step 3). To setup a webapi-project in a folder called lesson3 excute the following command: 

`dotnet new webapi -o lesson3`

Go to the folder of your project type those commands in the command line:

`dotnet add package Newtonsoft.Json`
`dotnet restore`

You can compile and run this project to check if the web API is working by default. After excuting the run-command `dotnet run` in the console your web API will start working by showing the message: 

`Now listening on: http://localhost:5000`
`Application started. Press Ctrl+C to shut down.`



###  Step 2: Web API check

- By default the .net core will generate a controller for you in the root folder. Go to the controller-folder and open the file `ValuesController.cs`. 

```
using System;
using System.Collections.Generic;
using System.Linq;
using System.Threading.Tasks;
using Microsoft.AspNetCore.Mvc;

namespace lesson3.Controllers
{
    //This is the default route of the API. 
    [Route("api/[controller]")]
    public class ValuesController : Controller
    {
        // GET api/values
        [HttpGet]
        public IEnumerable<string> Get()
        {
            return new string[] { "value1", "value2" };
        }

        // GET api/values/5
        [HttpGet("{id}")]
        public string Get(int id)
        {
            return "value";
        }

        // POST api/values
        [HttpPost]
        public void Post([FromBody]string value)
        {
        }

        // PUT api/values/5
        [HttpPut("{id}")]
        public void Put(int id, [FromBody]string value)
        {
        }

        // DELETE api/values/5
        [HttpDelete("{id}")]
        public void Delete(int id)
        {
        }
    }
}
```

- By default the web API will have the uri: `http://localhost:5000/api/values`. To check the communication between this API and a client we will use `Postman`. After starting `Postman` Select the command `GET`; enter the uri mentioned above; and click on `send-button`. After few seconds you should recieve a HTTP message from your api in json-format. the default message should be: 

`{"value1", "value2"}`.
 
### step 3: Add a model

Now we need to adapt the default controller (ValuesController.cs) to get it working with our data storage and the model from practicum1. 

- You need to make few adjustments to get that model working in your current project setup. 
    - Create a folder named `Models`
    - Copy the `Models.cs` file from your previous project (practicum1) to the `Models` folder of your current project. 
    - Change the namespace to match the following:

`namespace lesson3.Models`

- The files that needs to be adjusted from now on are: 
    - `Models.cs` in the Model folder
    - `Startup.cs` in the root directory of your project
    - `ValuesController` in the Controller folder 
 
- Change the class `MovieContext` in `Models.cs` to match this structure: 

```
...
namespace lesson3.Models
{
 public class MovieContext : DbContext
    {

        public DbSet<Actors> Actors { get; set; }
        public DbSet<Movies> Movies { get; set; }
        
        //Added constructor to provide the connection to the database as a service (look at: startup.cs)
        public MovieContext(DbContextOptions<MovieContext> options): base(options)
        {
        }
        
        //Uncomment or remove the OnConfiguring method. It is not need any more
        // protected override void OnConfiguring(DbContextOptionsBuilder optionsBuilder)
        // {
        //     if (!optionsBuilder.IsConfigured)
        //     {
        //
        //         // http://go.microsoft.com/fwlink/?LinkId=723263 for guidance on storing connection strings
        //         optionsBuilder.UseNpgsql(@"Host=localhost;Database=MovieDB;Username=postgres;Password=postgres");
        //     }
        // }
...
}
```

- The connection to the database will be provided as a service in this project. You need to add the connection string  to the `ConfigureServices` method in `Startup.cs`. Remember to change the connection parameters.

```
...

// This method gets called by the runtime. Use this method to add services to the container.
public void ConfigureServices (IServiceCollection services) {

        //Add this line to your method
     services.AddDbContext<MovieContext> (
                 opt => opt.UseNpgsql(@"Host=localhost;Database=MovieDB;Username=postgres;Password=postgres"));
     
     services.AddMvc ();

        }
...

```

- Change the name `ValuesController.cs` to `MoviesController.cs`.  We need to add a private property for the context to access it from our controller. This property will be intilized in the contructor. To understand how the context can be used in the controller the method `Get()` and `Get(int id)` are implemented to get the movies as a result form the database. 

```

using System;
using System.Collections.Generic;
using System.Linq;
using System.Linq.Expressions;
using System.Threading.Tasks;
using lesson3.Models;
using Microsoft.AspNetCore.Mvc;

namespace lesson3.Controllers {

    [Route ("api/[controller]")]
    public class MoviesController : Controller {
        private readonly MovieDBContext _context;

        public MoviesController (MovieDBContext context) {
            _context = context;
        }

        // GET api/values
        [HttpGet]
        public IEnumerable<Movies> Get () {
            return _context.Movies.ToList ();;
        }

        // GET api/values/5
        [HttpGet ("{id}")]
        public IActionResult Get (int id) {
            var movie = _context.Movies.FirstOrDefault (t => t.MovieId == id);
            if (movie == null) {
                return NotFound ();
            }
            return new ObjectResult (movie);
        }

        // POST api/values
        [HttpPost]
        public void Post ([FromBody] Movies value) { }

        // PUT api/values/5
        [HttpPut ("{id}")]
        public void Put (int id, [FromBody] Movies value) { }

        // DELETE api/values/5
        [HttpDelete ("{id}")]
        public void Delete (int id) { }
    }
}

```

